In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time

chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")  #open tab in incognito window
# chrome_options.add_experimental_option('detach',True)   #to hold the window on screen
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

driver = uc.Chrome(options=chrome_options)

driver.get('https://www.99acres.com/search/property/buy/residential-apartments/gurgaon?city=8&property_type=1&preference=S&area_unit=1&res_com=R')
time.sleep(4)

## select only resale flats
try:
    purchase_type = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//span[text()="Purchase type"]'))
    )

    # Scroll to the element
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", purchase_type)
    time.sleep(1)

    # Click it using JavaScript (safe for non-clickable spans)
    driver.execute_script("arguments[0].click();", purchase_type)
    print("✅ Clicked on 'Purchase type' span.")
    time.sleep(1)

    resale_btn = driver.find_element(By.XPATH , '//span[text() = "Resale"]')
    driver.execute_script("arguments[0].click();", resale_btn)
    time.sleep(2)

except Exception as e:
    print("Could not click purchase type':", e)


#extract links of resale flats
links = []
old_height = driver.execute_script('return document.body.scrollHeight')
count = 0

while True:
    time.sleep(2)

    ##scroll to bottom of page
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(2)

    new_height = driver.execute_script('return document.body.scrollHeight')

    print("old height" , old_height)
    print('new height' , new_height)

    if new_height == old_height:

        count+=1
        print('count : ',count)
        print("new height = old height")

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_links = soup.find_all('a', class_ = "tupleNew__propertyHeading ellipsis")
        for tag in page_links:
            href = tag.get('href')
            if href:
                links.append(href)
        print('total links = ', len(links))
        try:

            driver.execute_script("""const banner = document.getElementById('cs_PopUp');
                                  if (banner) banner.remove();""")  #remove any banner blocking click on next_button

            next_button = driver.find_element(By.XPATH, '//a[contains(text(), "Next")]') 
            # Scroll to make sure it’s visible
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(1)
            next_button.click()
            print("✅ Clicked on the next page.")

        except Exception as e:
            print("Could not click the next button:", e)
            break

    else:
        old_height = new_height
    
    if len(links)>=6000:
      break

with open('webscrape_links.txt', 'w') as f:
    f.write("\n".join(links))

# Pause the script to keep the browser open
input("Press ENTER to exit...")

KeyboardInterrupt: 

In [ ]:
#scrape each page - main

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

#Load links from file
all_links = []
with open('webscrape_links_v2.txt', 'r') as file:
    for line in file:
        all_links.append(line.strip())

# Set up Chrome
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--incognito")
driver = uc.Chrome(options=options)


# df = pd.DataFrame(columns = ['address','price','super_area'])
df = pd.DataFrame(columns = ['link','address','price','super_area', 'bedrooms','bathroom','balcony','additional_rooms','age_possession','floor','furnishing_type','facing','num_parking','features'])

count = 0
for i in range(5800,len(all_links)):
    count+=1
    print(f"Scraping {all_links[i]}")
    print('link number',count)

    details = {}
    details['link'] = all_links[i]
    try:
        driver.get(all_links[i])
        time.sleep(3)  # allow page to load

        #extract address
        try:
            details['address'] = driver.find_element(by = By.XPATH , value = '//*[@id="FactTableComponent"]/tbody/tr[2]/td[2]/div[2]').text.strip()
            #//*[@id="FloatingHeaderComponentTop"]/div/div[2]/div[1]/h1/div[2]/span
        except Exception as e:
            print('Unable to extract address -> error ', e)
            details['address'] = np.nan

        #extract price
        try:
            details['price'] = driver.find_element(by = By.XPATH , value = '//*[@id="pdPrice"]').text.strip()
        except Exception as e:
            print('Unable to extract price -> error ', e)
            details['price'] = np.nan
        
        #super area
        try:
            details['super_area'] = driver.find_element(by = By.XPATH , value = '//*[@id="superbuiltupArea_span"]').text.strip()

        except Exception as e:
            try:    #if carpet area is given
                details['super_area'] = float(driver.find_element(by = By.XPATH , value = '//*[@id="carpetArea_span"]').text.strip())*1.42
                print('scaped carpet area')
            except Exception as e:
                try:    #if built up area is given
                    
                    details['super_area'] = float(driver.find_element(by = By.XPATH , value = '//*[@id="builtupArea_span"]').text.strip())*1.25
                    print('scaped built up area')
                except Exception as e:
                    print('Unable to extract area -> error ', e)
                    details['super_area'] = np.nan

        #bedrooms
        try:
            details['bedrooms'] = driver.find_element(by = By.XPATH , value = '//*[@id="bedRoomNum"]').text.strip()
        except Exception as e:
            print('Unable to extract bedrooms -> error ', e)
            details['bedrooms'] = np.nan

        #bathroom
        try:
            details['bathroom'] = driver.find_element(by = By.XPATH , value = '//*[@id="bathroomNum"]').text.strip()
        except Exception as e:
            print('Unable to extract bathroom -> error ', e)
            details['bathroom'] = np.nan

        #balcony
        try:
            details['balcony'] = driver.find_element(by = By.XPATH , value = '//*[@id="balconyNum"]').text.strip()
        except Exception as e:
            print('Unable to extract balcony -> error ', e)
            details['balcony'] = np.nan

        #additional_rooms
        try:
            details['additional_rooms'] = driver.find_element(by = By.XPATH , value = '//*[@id="additionalRooms"]').text.strip()
        except Exception as e:
            print('Unable to extract additional_rooms -> error ', e)
            details['additional_rooms'] = np.nan

        #age_possession
        try:
            details['age_possession'] = driver.find_element(by = By.XPATH , value = '//*[@id="agePossessionLbl"]').text.strip()
        except Exception as e:
            print('Unable to extract age_possession -> error ', e)
            details['age_possession'] = np.nan
        
        #floor
        try:
            details['floor'] = driver.find_element(by = By.XPATH , value = '//*[@id="floorNumLabel"]').text.strip()
        except Exception as e:
            print('Unable to extract floor -> error ', e)
            details['floor'] = np.nan

        #'furnishing_type'
        try:
            details['furnishing_type'] = driver.find_element(by = By.XPATH , value = '//*[@id="Furnish_Label"]').text.strip()
        except Exception as e:
            print('Unable to extract furnishing_type -> error ', e)
            details['furnishing_type'] = np.nan
        
        #facing
        try:
            details['facing'] = driver.find_element(by = By.XPATH , value = '//*[@id="facingLabel"]').text.strip()
        except Exception as e:
            print('Unable to extract facing -> error ', e)
            details['facing'] = np.nan
        
        #num_parking
        try:
            details['num_parking'] = driver.find_element(by = By.XPATH , value = '//*[@id="Reserved_Parking_Label"]').text.strip()
        except Exception as e:
            print('Unable to extract num_parking -> error ', e)
            details['num_parking'] = np.nan

        #features
        try:
            features_div = driver.find_element(By.XPATH, '//div[@data-label="FACILITIES" and contains(@class, "component__features")]')

            # find all feature <div> elements inside the <li> tags
            li_elements = features_div.find_elements(By.TAG_NAME, 'li')

            features = []
            for li in li_elements:
                try:
                    feature_text = li.find_element(By.TAG_NAME, 'div').text.strip()
                    if feature_text:
                        features.append(feature_text)
                except:
                    continue
            
            details['features'] = features

        except Exception as e:
            print('Unable to extract features -> error ', e)
            details['features'] = np.nan
                
        #create dataframe
        df = pd.concat([df,pd.DataFrame([details])],ignore_index= True)

    except Exception as e:
        print(f"❌ Error scraping {all_links[i]}: {e}")

df.to_csv('gurgaon_flats_from_245.csv',index= False)

driver.quit()


Scraping https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-sale-in-eldeco-accolade-sohna-gurgaon-1200-sq-ft-spid-D79622507
link number 1
Unable to extract additional_rooms -> error  Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="additionalRooms"]"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   undetected_chromedriver             0x000000010115d448 undetected_chromedriver + 6149192
1   undetected_chromedriver             0x0000000101154b0a undetected_chromedriver + 6114058
2   undetected_chromedriver             0x0000000100be5e80 undetected_chromedriver + 417408
3   undetected_chromedriver             0x0000000100c37587 undetected_chromedriver + 750983
4   undetected_chromedriver             0x0000000100c377a1 undetected_chromedriver + 751521
5   undetected_chromedriver             0

IndexError: list index out of range

In [2]:
#scrape remaining pages - main

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

#Load links from file
all_links = []
with open('remaining_links.txt', 'r') as file:
    for line in file:
        all_links.append(line.strip())

# Set up Chrome
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--incognito")
driver = uc.Chrome(options=options)


# df = pd.DataFrame(columns = ['address','price','super_area'])
df = pd.DataFrame(columns = ['link','address','price','super_area', 'bedrooms','bathroom','balcony','additional_rooms','age_possession','floor','furnishing_type','facing','num_parking','features'])

# all_links = [
#     'https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-sale-in-vipul-lavanya-sector-81-gurgaon-1783-sq-ft-spid-Z81401399']

count = 0
for i in range(250,len(all_links)):
    count+=1
    print(f"Scraping {all_links[i]}")
    print('link number',count)

    details = {}
    details['link'] = all_links[i]
    try:
        driver.get(all_links[i])
        time.sleep(3)  # allow page to load

        #extract address
        try:
            details['address'] = driver.find_element(by = By.XPATH , value = '//*[@id="FactTableComponent"]/tbody/tr[2]/td[2]/div[2]').text.strip()
            #//*[@id="FloatingHeaderComponentTop"]/div/div[2]/div[1]/h1/div[2]/span
        except Exception as e:
            print('Unable to extract address -> error ', e)
            details['address'] = np.nan

        #extract price
        try:
            details['price'] = driver.find_element(by = By.XPATH , value = '//*[@id="pdPrice"]').text.strip()
        except Exception as e:
            print('Unable to extract price -> error ', e)
            details['price'] = np.nan
        
        #super area
        try:
            details['super_area'] = driver.find_element(by = By.XPATH , value = '//*[@id="superbuiltupArea_span"]').text.strip()

        except Exception as e:
            try:    #if carpet area is given
                details['super_area'] = float(driver.find_element(by = By.XPATH , value = '//*[@id="carpetArea_span"]').text.strip())*1.42
                print('scaped carpet area')
            except Exception as e:
                try:    #if built up area is given
                    
                    details['super_area'] = float(driver.find_element(by = By.XPATH , value = '//*[@id="builtupArea_span"]').text.strip())*1.25
                    print('scaped built up area')
                except Exception as e:
                    print('Unable to extract area -> error ', e)
                    details['super_area'] = np.nan

        #bedrooms
        try:
            details['bedrooms'] = driver.find_element(by = By.XPATH , value = '//*[@id="bedRoomNum"]').text.strip()
        except Exception as e:
            print('Unable to extract bedrooms -> error ', e)
            details['bedrooms'] = np.nan

        #bathroom
        try:
            details['bathroom'] = driver.find_element(by = By.XPATH , value = '//*[@id="bathroomNum"]').text.strip()
        except Exception as e:
            print('Unable to extract bathroom -> error ', e)
            details['bathroom'] = np.nan

        #balcony
        try:
            details['balcony'] = driver.find_element(by = By.XPATH , value = '//*[@id="balconyNum"]').text.strip()
        except Exception as e:
            print('Unable to extract balcony -> error ', e)
            details['balcony'] = np.nan

        #additional_rooms
        try:
            details['additional_rooms'] = driver.find_element(by = By.XPATH , value = '//*[@id="additionalRooms"]').text.strip()
        except Exception as e:
            print('Unable to extract additional_rooms -> error ', e)
            details['additional_rooms'] = np.nan

        #age_possession
        try:
            details['age_possession'] = driver.find_element(by = By.XPATH , value = '//*[@id="agePossessionLbl"]').text.strip()
        except Exception as e:
            print('Unable to extract age_possession -> error ', e)
            details['age_possession'] = np.nan
        
        #floor
        try:
            details['floor'] = driver.find_element(by = By.XPATH , value = '//*[@id="floorNumLabel"]').text.strip()
        except Exception as e:
            print('Unable to extract floor -> error ', e)
            details['floor'] = np.nan

        #'furnishing_type'
        try:
            details['furnishing_type'] = driver.find_element(by = By.XPATH , value = '//*[@id="Furnish_Label"]').text.strip()
        except Exception as e:
            print('Unable to extract furnishing_type -> error ', e)
            details['furnishing_type'] = np.nan
        
        #facing
        try:
            details['facing'] = driver.find_element(by = By.XPATH , value = '//*[@id="facingLabel"]').text.strip()
        except Exception as e:
            print('Unable to extract facing -> error ', e)
            details['facing'] = np.nan
        
        #num_parking
        try:
            details['num_parking'] = driver.find_element(by = By.XPATH , value = '//*[@id="Reserved_Parking_Label"]').text.strip()
        except Exception as e:
            print('Unable to extract num_parking -> error ', e)
            details['num_parking'] = np.nan

        #features
        try:
            features_div = driver.find_element(By.XPATH, '//div[@data-label="FACILITIES" and contains(@class, "component__features")]')

            # find all feature <div> elements inside the <li> tags
            li_elements = features_div.find_elements(By.TAG_NAME, 'li')

            features = []
            for li in li_elements:
                try:
                    feature_text = li.find_element(By.TAG_NAME, 'div').text.strip()
                    if feature_text:
                        features.append(feature_text)
                except:
                    continue
            
            details['features'] = features

        except Exception as e:
            print('Unable to extract features -> error ', e)
            details['features'] = np.nan
                
        #create dataframe
        df = pd.concat([df,pd.DataFrame([details])],ignore_index= True)

    except Exception as e:
        print(f"❌ Error scraping {all_links[i]}: {e}")

df.to_csv('gurgaon_flats_from_245.csv',index= False)

driver.quit()


Scraping https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-sale-in-m3m-merlin-sector-67-gurgaon-2427-sq-ft-spid-O80176113
link number 1
Unable to extract address -> error  Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="FactTableComponent"]/tbody/tr[2]/td[2]/div[2]"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   undetected_chromedriver             0x000000010121b448 undetected_chromedriver + 6149192
1   undetected_chromedriver             0x0000000101212b0a undetected_chromedriver + 6114058
2   undetected_chromedriver             0x0000000100ca3e80 undetected_chromedriver + 417408
3   undetected_chromedriver             0x0000000100cf5587 undetected_chromedriver + 750983
4   undetected_chromedriver             0x0000000100cf57a1 undetected_chromedriver + 751521
5   undetected_chromedr

In [ ]:
#scrape each page -test

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

#Load links from file
# all_links = []
# with open('webscrape_links_v2.txt', 'r') as file:
#     for line in file:
#         all_links.append(line.strip())

# Set up Chrome
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--incognito")
driver = uc.Chrome(options=options)


# df = pd.DataFrame(columns = ['address','price','super_area'])
df = pd.DataFrame(columns = ['link','address','price','super_area', 'bedrooms','bathroom','balcony','additional_rooms','age_possession','floor','furnishing_type','facing','num_parking','features'])

all_links = [
    'https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-sale-in-vipul-lavanya-sector-81-gurgaon-1783-sq-ft-spid-Z81401399']

count = 0
for link in all_links:
    count+=1
    print(f"Scraping {link}")
    print('link number',count)

    details = {}
    details['link'] = link
    try:
        driver.get(link)
        time.sleep(3)  # allow page to load

        #extract address
        try:
            details['address'] = driver.find_element(by = By.XPATH , value = '//*[@id="FactTableComponent"]/tbody/tr[2]/td[2]/div[2]').text.strip()
            #//*[@id="FloatingHeaderComponentTop"]/div/div[2]/div[1]/h1/div[2]/span
        except Exception as e:
            print('Unable to extract address -> error ', e)
            details['address'] = np.nan

        #extract price
        try:
            details['price'] = driver.find_element(by = By.XPATH , value = '//*[@id="pdPrice"]').text.strip()
        except Exception as e:
            print('Unable to extract price -> error ', e)
            details['price'] = np.nan
        
        #super area
        try:
            details['super_area'] = driver.find_element(by = By.XPATH , value = '//*[@id="superbuiltupArea_span"]').text.strip()

        except Exception as e:
            try:    #if carpet area is given
                details['super_area'] = float(driver.find_element(by = By.XPATH , value = '//*[@id="carpetArea_span"]').text.strip())*1.42
                print('scaped carpet area')
            except Exception as e:
                try:    #if built up area is given
                    
                    details['super_area'] = float(driver.find_element(by = By.XPATH , value = '//*[@id="builtupArea_span"]').text.strip())*1.25
                    print('scaped built up area')
                except Exception as e:
                    print('Unable to extract area -> error ', e)
                    details['super_area'] = np.nan

        #bedrooms
        try:
            details['bedrooms'] = driver.find_element(by = By.XPATH , value = '//*[@id="bedRoomNum"]').text.strip()
        except Exception as e:
            print('Unable to extract bedrooms -> error ', e)
            details['bedrooms'] = np.nan

        #bathroom
        try:
            details['bathroom'] = driver.find_element(by = By.XPATH , value = '//*[@id="bathroomNum"]').text.strip()
        except Exception as e:
            print('Unable to extract bathroom -> error ', e)
            details['bathroom'] = np.nan

        #balcony
        try:
            details['balcony'] = driver.find_element(by = By.XPATH , value = '//*[@id="balconyNum"]').text.strip()
        except Exception as e:
            print('Unable to extract balcony -> error ', e)
            details['balcony'] = np.nan

        #additional_rooms
        try:
            details['additional_rooms'] = driver.find_element(by = By.XPATH , value = '//*[@id="additionalRooms"]').text.strip()
        except Exception as e:
            print('Unable to extract additional_rooms -> error ', e)
            details['additional_rooms'] = np.nan

        #age_possession
        try:
            details['age_possession'] = driver.find_element(by = By.XPATH , value = '//*[@id="agePossessionLbl"]').text.strip()
        except Exception as e:
            print('Unable to extract age_possession -> error ', e)
            details['age_possession'] = np.nan
        
        #floor
        try:
            details['floor'] = driver.find_element(by = By.XPATH , value = '//*[@id="floorNumLabel"]').text.strip()
        except Exception as e:
            print('Unable to extract floor -> error ', e)
            details['floor'] = np.nan

        #'furnishing_type'
        try:
            details['furnishing_type'] = driver.find_element(by = By.XPATH , value = '//*[@id="Furnish_Label"]').text.strip()
        except Exception as e:
            print('Unable to extract furnishing_type -> error ', e)
            details['furnishing_type'] = np.nan
        
        #facing
        try:
            details['facing'] = driver.find_element(by = By.XPATH , value = '//*[@id="facingLabel"]').text.strip()
        except Exception as e:
            print('Unable to extract facing -> error ', e)
            details['facing'] = np.nan
        
        #num_parking
        try:
            details['num_parking'] = driver.find_element(by = By.XPATH , value = '//*[@id="Reserved_Parking_Label"]').text.strip()
        except Exception as e:
            print('Unable to extract num_parking -> error ', e)
            details['num_parking'] = np.nan

        #features
        try:
            features_div = driver.find_element(By.XPATH, '//div[@data-label="FACILITIES" and contains(@class, "component__features")]')

            # find all feature <div> elements inside the <li> tags
            li_elements = features_div.find_elements(By.TAG_NAME, 'li')

            features = []
            for li in li_elements:
                try:
                    feature_text = li.find_element(By.TAG_NAME, 'div').text.strip()
                    if feature_text:
                        features.append(feature_text)
                except:
                    continue
            
            details['features'] = features

        except Exception as e:
            print('Unable to extract features -> error ', e)
            details['features'] = np.nan
                
        #create dataframe
        df = pd.concat([df,pd.DataFrame([details])],ignore_index= True)

    except Exception as e:
        print(f"❌ Error scraping {link}: {e}")

df.to_csv('gurgaon_flats.csv',index= False)

driver.quit()


Scraping https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-sale-in-vipul-lavanya-sector-81-gurgaon-1783-sq-ft-spid-Z81401399
link number 1
scaped built up area


/var/folders/s_/8psmrzpd1ljgqhthwdx4fv000000gn/T/ipykernel_73053/2320502380.py:158: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame([details])],ignore_index= True)
